In [4]:
import gensim
import pandas as pd
import jieba
import re
import stanfordcorenlp
import math
import numpy as np
import random

In [7]:
path='D:/Github/AIClass/datasource_lecutre/datasource/export_sql_1558435/'

In [8]:
data_source = path +"sqlResult_1558435.csv"

In [9]:
data = pd.read_csv(data_source,encoding='UTF-8')

In [10]:
data = data.fillna('')

In [11]:
def cut(string): return ' '.join(jieba.cut(string))

In [12]:
cut("你今天运动了吗")

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\sdm\AppData\Local\Temp\jieba.cache
Loading model cost 2.082 seconds.
Prefix dict has been built succesfully.


'你 今天 运动 了 吗'

In [13]:
content = data['content'].tolist()

In [14]:
def token(string):
    return re.findall(r'[\d|\w]+',string)

In [15]:
token("这是一个测试\n\n\n")

['这是一个测试']

In [16]:
news_content = [token(n) for n in content]

In [17]:
news_content = [''.join(n) for n in news_content]

In [18]:
news_content = [cut(n) for n in news_content]

In [19]:
news_content[:2]

['此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息',
 '骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台']

In [20]:
len(news_content)

89611

In [21]:
def document_frequency(word):
    return sum(1 for n in news_content if word in n)

In [22]:
def idf(word):
    return math.log10(len(news_content)/document_frequency(word))

In [23]:
idf("的")

0.1051466115514474

In [25]:
idf("京东")

2.6601052524359976

In [26]:
def tf(word,document):
    words = document.split()
    return sum(1 for w in words if w==word)

In [28]:
tf("小米",news_content[0])

1

In [29]:
def tf_idf(word,document):
    return tf(word,document)*idf(word)

In [30]:
tf_idf("的",news_content[0])

0.2102932231028948

In [31]:
def get_keywords_of_a_document(document):
    words = set(document.split())
    tfidf = [
        (w,tf_idf(w,document)) for w in words
    ]
    tfidf = sorted(tfidf,key=lambda x: x[1],reverse=True)
    return tfidf

In [33]:
get_keywords_of_a_document(news_content[2])

[('一加', 14.083990238533945),
 ('279928992999', 4.9523613237924735),
 ('5V4A', 4.9523613237924735),
 ('DashCharge', 4.9523613237924735),
 ('xx99', 4.9523613237924735),
 ('3400mAh', 4.9523613237924735),
 ('6G8GBRAM64GB128GBROM', 4.9523613237924735),
 ('电池', 4.897141281470585),
 ('1080P', 4.651331328128492),
 ('刘作', 4.475240069072811),
 ('3T', 4.253391319456455),
 ('快充', 4.17421007340883),
 ('备货', 3.838417971485637),
 ('AMOLED', 3.776270064736792),
 ('轻薄', 3.7219124024142),
 ('手机', 3.4361943988273684),
 ('市面', 3.095028827361205),
 ('缩水', 3.0831296040614973),
 ('显示屏', 3.038547471408757),
 ('大屏', 2.9523613237924735),
 ('旗舰', 2.8554513107844173),
 ('某个', 2.7270520420666107),
 ('摄像头', 2.716832876884925),
 ('1600', 2.704388057430667),
 ('规格', 2.6828483795745575),
 ('京东', 2.6601052524359976),
 ('三星', 2.6556961335309426),
 ('泄露', 2.604056460744313),
 ('惊喜', 2.5526876023114355),
 ('所说', 2.513028629962211),
 ('售价', 2.4513020615747223),
 ('设定', 2.4312232400884373),
 ('寸', 2.3912599401434176),
 ('搭载

In [34]:
news_content[2]

'此前 的 一加 3T 搭载 的 是 3400mAh 电池 DashCharge 快充 规格 为 5V4A 至于 电池 缩水 可能 与 刘作 虎 所说 一加 手机 5 要 做 市面 最 轻薄 大屏 旗舰 的 设定 有关 按照 目前 掌握 的 资料 一加 手机 5 拥有 55 寸 1080P 三星 AMOLED 显示屏 6G8GBRAM64GB128GBROM 双 1600 万 摄像头 备货 量 惊喜 根据 京东 泄露 的 信息 一加 5 起 售价 是 xx99 元 应该 是 在 279928992999 中 的 某个'

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [36]:
vectorizer = TfidfVectorizer(max_features=10000)

In [37]:
sample_num = 50000
sub_samples = news_content[:sample_num]

In [38]:
X = vectorizer.fit_transform(sub_samples)

In [39]:
len(vectorizer.vocabulary_)

10000

In [40]:
np.where(X[1].toarray())  # find positions whose values is not zero 

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64),
 array([ 100,  281,  360,  372,  472,  578, 1008, 1162, 1244, 1269, 1285,
        1380, 1462, 1886, 2435, 2754, 2760, 2989, 3067, 3353, 3800, 3990,
        4173, 4272, 4429, 4497, 4521, 4529, 4595, 4658, 4961, 5037, 5104,
        5136, 5550, 5717, 5771, 5899, 6035, 6081, 6710, 6971, 7072, 7217,
        7506, 7986, 8025, 8026, 8137, 8207, 8479, 8558, 8799, 9127, 9196,
        9252, 9637, 9744, 9824], dtype=int64))

In [44]:
X[1].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [45]:
len(X[1].toarray())

1

In [46]:
document_1,document_2 = random.randint(0,1000),random.randint(0,1000)

In [47]:
news_content[document_1]

'刘国梁 新华社 北京 6 月 20 日电 记者 从 中国乒乓球协会 获悉 国家乒乓球队 管理模式 将 进行 改革 不再 设 总 主教练 岗位 设置 设 男子 女子 两个 国家队教练组 一手 抓 竞技 训练 一手 抓 队伍 管理 两手抓 两手 都 要 硬 减少 管理层 级 推动 国家队 实现 扁平化 管理 提高 训练 备战 效率 和 效益 据悉 为 深化 体育 改革 创新 体制 机制 积极 应对 2020 东京 奥运会 新 的 挑战 实现 运动 成绩 和 精神文明 双丰收 中国乒乓球协会 根据 近 阶段 体育 社团 改革 的 需要 瞄准 国家队 建设 的 方向 针对 存在 的 一些 问题 和 薄弱环节 推出 了 系列 改革 措施 另一方面 积极 充分调动 和 发挥 人才 优势 进一步 突出 和 强化 协会 功能 经 中国乒乓球协会 代表大会 常务委员会 讨论 决定 刘国梁 担任 中国乒乓球协会 副 主席 同时 雷军 接替 刘晓农 担任 中国乒乓球协会 副 主席 兼 秘书长 据悉 针对 日前 广受 社会 关注 的 孔令辉 被 新加坡 一间 赌场 追债 一事 体育总局 乒乓球 羽毛球 运动 管理中心 党委 正在 根据 相关 规定 和 程序 对 相关 违纪 线索 进行 核实 并 将 依据 党规 党纪 做出 严肃处理 通过 此举 塑造 风清气 正 的 良好环境'

In [48]:
vector_of_document_1 = X[document_1].toarray()[0]

In [49]:
vector_of_document_2 = X[document_2].toarray()[0]

In [51]:
from scipy.spatial.distance import cosine
def distance(v1,v2): return cosine(v1,v2)

In [52]:
distance(vector_of_document_1,vector_of_document_2)

0.9430060197077451

## LDA

In [53]:
from gensim import corpora, models
import jieba.posseg as jp,jieba

In [54]:
news = news_content[:100]

In [55]:
flags = ('n', 'nr', 'ns', 'nt', 'eng', 'v', 'd')

In [57]:
stop_words = []
with open(path + '百度停用词表.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

In [58]:
words_ls = []
for text in news:
    words = [w.word for w in jp.cut(text) if w.flag in flags and w.word not in stop_words]
    words_ls.append(words)

In [59]:
dictionary = corpora.Dictionary(words_ls)

In [60]:
corpus = [dictionary.doc2bow(words) for words in words_ls]

In [61]:
lda = models.ldamodel.LdaModel(corpus=corpus,id2word=dictionary,num_topics=5)

In [62]:
for topic in lda.print_topics(num_words=10):
    print(topic)

(0, '0.005*"市场" + 0.004*"都" + 0.004*"索尼" + 0.004*"内容" + 0.004*"还" + 0.003*"不" + 0.003*"会" + 0.003*"企业" + 0.003*"中国" + 0.003*"PS4"')
(1, '0.006*"企业" + 0.005*"跨境" + 0.005*"都" + 0.004*"人" + 0.004*"配送" + 0.004*"会" + 0.004*"无人" + 0.004*"不" + 0.004*"摄像头" + 0.003*"还"')
(2, '0.006*"不" + 0.005*"会" + 0.005*"记者" + 0.004*"市场" + 0.004*"都" + 0.004*"中国" + 0.003*"板块" + 0.003*"人" + 0.003*"还" + 0.003*"企业"')
(3, '0.007*"市场" + 0.006*"会" + 0.006*"中国" + 0.006*"都" + 0.005*"人" + 0.005*"手机" + 0.005*"不" + 0.004*"还" + 0.004*"冰架" + 0.003*"乐视"')
(4, '0.006*"市场" + 0.006*"不" + 0.005*"还" + 0.005*"都" + 0.004*"人" + 0.004*"会" + 0.003*"中国" + 0.003*"更" + 0.003*"国家" + 0.003*"相关"')


In [64]:
news[1]

'骁龙 835 作为 唯一 通过 Windows10 桌面 平台 认证 的 ARM 处理器 高通 强调 不会 因为 只 考虑 性能 而 去 屏蔽掉 小 核心 相反 他们 正 联手 微软 找到 一种 适合 桌面 平台 的 兼顾 性能 和 功耗 的 完美 方案 报道 称 微软 已经 拿到 了 一些 新 的 源码 以便 Windows10 更好 地 理解 biglittle 架构 资料 显示 骁龙 835 作为 一款 集成 了 CPUGPU 基带 蓝牙 WiFi 的 SoC 比 传统 的 Wintel 方案 可以 节省 至少 30 的 PCB 空间 按计划 今年 Q4 华硕 惠普 联想 将 首发 骁龙 835Win10 电脑 预计 均 是 二合一 形态 的 产品 当然 高通 骁龙 只是 个 开始 未来 也许 还 能 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 等 进入 Windows10 桌面 平台'

### 3.1 手动实现TextRank算法 (在新闻数据中随机提取100条新闻训练词向量和做做法测试）

In [70]:
model = gensim.models.Word2Vec(words_ls, min_count=1)

In [71]:
from collections import defaultdict
from scipy.spatial.distance import cosine
from icecream import ic

In [98]:
def creat_graph(words_list, window_size=3):
    words_list = list(set(words_list))
    length = len(words_list)
    edge_weights = {}
    out = {}
    word_index = {}#记录词的index
    index_dict = {}#记录节点index对应的词
    for i, w in enumerate(words_list):
        index_dict[i] = w
        word_index[w] = i
        start = i - window_size + 1
        if start < 0:
            start = 0
        end = i + window_size
        if end > length:
            end = length
        out[w] = words_list[start:i]+words_list[i+1:end]
        
        for j in range(1,window_size):
            if (i+j)<length:
                edge_weights[(w,words_list[i+j])] = cosine(model[w],model[words_list[i+j]])
                edge_weights[(words_list[i+j],w)] = cosine(model[w],model[words_list[i+j]])
    
    return edge_weights,out,index_dict,word_index

In [122]:
def TextRank_get_kws(topK, words_list, d=0.85, iternum = 200):
    edge_w, out, index_dict, word_index= creat_graph(words_list)
    words_list = list(set(words_list))
    score = np.ones([len(words_list)])
    score /= (len(words_list))
    sum_out = 0
    sum_in = 0
    word_pr = {}
    for n in range(iternum):
        for i in range(len(words_list)):
            for e1 in out[words_list[i]]:
                sum_out = sum(edge_w[(e1,e2)] for e2 in out[e1])
                sum_in += edge_w[(words_list[i],e1)] * score[word_index[e1]]/sum_out
            score[i] = (1-d) + d * sum_in
            
    for i in range(len(score)):
        word_pr[index_dict[i]] = score[i]
   
    return [w for w,_ in sorted(word_pr.items(), reverse = True)][:topK]

In [127]:
news[0]

'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

In [126]:
words_ls[0]

['小米',
 '手机',
 '机型',
 '机型',
 '已',
 '暂停',
 '更新',
 '发布',
 '含',
 '开发',
 '版',
 '体验版',
 '内测',
 '稳定版',
 '暂不',
 '受',
 '确保',
 '工程师',
 '精力',
 '系统优化',
 '精力',
 '用到',
 'MIUI9',
 'MIUI8',
 '发布',
 '距今已有',
 'MIUI9',
 '信息',
 '等待',
 '官方消息']

In [125]:
kw =TextRank_get_kws(5,words_ls[0])
print(kw)

['距今已有', '系统优化', '精力', '等待', '稳定版']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in double_scalars
  del sys.path[0]


### 3.2 使用词向量和k-means的方法寻找关键词

In [128]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cosine
from collections import defaultdict

In [129]:
def get_kw_km_cls(text, n_class):
    text = list(set(text))

    sen_vec = [model[w] for w in text]

    clf = KMeans(n_clusters=n_class, random_state=0)
    prediction = clf.fit_predict(sen_vec)
    
    vec_center = clf.cluster_centers_ #聚类中心   
    classes = defaultdict(list)
    similar = defaultdict(list)
    key_words = defaultdict(list)
    
    for label, word in zip(clf.labels_, text):
        classes[label].append(word)
        sim = (word, cosine(model[word],vec_center[label]))
        similar[label].append(sim)

    #每类取3个关键词
    for k, v in similar.items():
        key_words[k] = [w for w,_ in sorted(v, key = lambda x:x[1], reverse = True)[:3]] if len(v) > 3 else [w for w,_ in v]

    return classes, key_words

In [132]:
get_kw_km_cls(words_ls[0],2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


(defaultdict(list,
             {0: ['体验版',
               'MIUI8',
               '开发',
               '信息',
               '距今已有',
               '机型',
               '暂停',
               '官方消息',
               '确保',
               '内测',
               '受',
               '工程师',
               'MIUI9',
               '暂不',
               '稳定版',
               '含',
               '发布',
               '已',
               '版',
               '更新',
               '系统优化',
               '小米',
               '等待',
               '用到',
               '精力'],
              1: ['手机']}),
 defaultdict(list, {0: ['精力', '确保', 'MIUI8'], 1: ['手机']}))